In [1]:
import requests
headers = {
'Host':'tms.bote.taipei.gov.tw',
'Referer':'http://tms.bote.taipei.gov.tw/ttc2/main.jsp',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
'X-Requested-With':'XMLHttpRequest'
}

url = 'http://tms.bote.taipei.gov.tw/ttc2/vd.do?action=returnXML&vdKmlSrc=c:/Program%20Files/Apache%20Software%20Foundation/Tomcat%205.5/webapps/ttc2/kml/newVd.kml&_=1446635335037'
cre = requests.get(url,headers = headers)


In [ ]:
kml2data(res)

In [ ]:
data

In [1]:

import re
import pandas as pd
def kml2data(res):
    rawData=res.text.encode('utf8')
    trimHead = re.search(('(.*?@@)(.*)'),rawData)
    if trimHead is None:
        print rawData
        trimHead = expsolve()
    googlebye = trimHead.group(2)
    ary=[]
    dic = {'pk':'','address':'','lon':'','lat':'','carport':'','totalvol':'','avgspeed':''}
    #k[0],k[2], k[3], k[4]
    for sample in googlebye.split('^^%%'):
        k = sample.split('^^')
        k1= k[0].split(')')
        pk = k1[0][1:]
        address = k1[1]
        lon = k[3]
        lat = k[4]
        #print address
        #print lon,lat
        k2 = k[5].split('<br>')
        m = re.search('\d+',k2[0])
        m1 = re.search('\d+',k2[1])
        m2 = re.search('\d+',k2[2])
#         print k2[2]
#         print '車道:' + m.group(0)
#         print '平均流量:'+ m1.group(0) #k[5].split('<br>')[1]
#         print '速度:' + m2.group(0)
        dic = {'pk':pk,'address':address,'lon':lon,'lat':lat,'carport':m.group(0),'totalvol':m1.group(0),'avgspeed':m2.group(0)}
        ary.append(dic)
    data=pd.DataFrame(ary)
    return data


In [2]:
def expsolve():
    res = requests.get(url,headers = headers)
    rawData=res.text.encode('utf8')
    trimHead = re.search(('(.*?@@)(.*)'),rawData)
    if trimHead is None:
        return expsolve()
    return trimHead


In [ ]:
import os
import requests
import time
import datetime
import pandas as pd
from StringIO import StringIO

headers = {
        'Host':'tms.bote.taipei.gov.tw',
        'Referer':'http://tms.bote.taipei.gov.tw/ttc2/main.jsp',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
        'X-Requested-With':'XMLHttpRequest'
        }
old = datetime.datetime.now()
#old = datetime.datetime(2015, 11, 5, 13, 10, 35, 192000)
new = datetime.datetime.now()
#print type((new.date()-old.date()).days)
datename = str(new.date())
path = 'E:\\bitbucket\\dataclean\\hsu\\data\\carvol-{}.csv'.format(datename)
while True:
    try:
        url = 'http://tms.bote.taipei.gov.tw/ttc2/vd.do?action=returnXML&vdKmlSrc=c:/Program%20Files/Apache%20Software%20Foundation/Tomcat%205.5/webapps/ttc2/kml/newVd.kml&_=1446635335037'
        res = requests.get(url,headers = headers)
    except:
        print 'internet maybe some problem try again'
    #correct time miniute sync
    carvol = kml2data(res)
    new = datetime.datetime.now()
    sleeptime = 300 - (new.second + new.microsecond/1000000.0)
    delta = new.date()-old.date()
    if delta.days == 1:
        datename = str(new.date())
        path = 'E:\\bitbucket\\dataclean\\hsu\\data\\carvol-{}.csv'.format(datename)
    if not os.path.exists(path):
        with open (path,'w') as f:
            f.write('address,avgspeed,carport,lat,lon,pk,totalvol,time\n')
    
    #print sleeptime        
    carvol['time'] = new
    with open(path, 'a') as f:
        carvol.to_csv(f,encoding='utf-8',index=False,header=False)
    time.sleep(sleeptime)
    old = new 
